In [1]:
import sys, os, django
import pandas as pd
from django.core.exceptions import MultipleObjectsReturned, ObjectDoesNotExist

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from scoping.tasks import *
import time

In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupyter-google-e4b72bf44320.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1Z_BRPPW0s6i9JxFG-v5dRjHCXTDVHY2Qbgz4qMLGQ8c'
wb = gc.open_by_key(spreadsheet_key)
ws = wb.worksheet("Documents")

In [3]:
pid = 127

columns = {
    'id':'ID',
    'PY': 'Year',
    'authors':'Authors',
    'title':'Title',
    'wosarticle__di':'DOI'
}

reldocs = Doc.objects.filter(
    docproject__project_id=pid, docproject__relevant=1
).values_list('pk',flat=True)

added_docs = Doc.objects.filter(
    query__id__in=[7135,7242]
).values_list('pk',flat=True)

all_ids = set(reldocs) | set(added_docs)


docs = Doc.objects.filter(pk__in=all_ids)

df = pd.DataFrame.from_dict(list(docs.values(*columns.keys())))
df = df[columns.keys()]

df = df.rename(columns=columns)

def is_manually_added(row):
    d = Doc.objects.get(pk=row['ID'])
    qs = d.query.filter(project_id=pid, qtype="MN")
    return qs.exists()

df['manaully_added'] = df.apply(is_manually_added, axis=1)

print(df.shape)
df.head()

(816, 6)


,ID,Year,Authors,Title,DOI,manaully_added
0,10579,2016,"Betsill, M, Stevis, D",The politics and dynamics of energy transition...,10.1177/0263774X15614668,False
1,34283,2008,"Sheehan, P, Sheehan, P",The new global growth path: implications for c...,10.1007/s10584-008-9415-z,False
2,60066,2001,"Pierret, C.",For a restrained or controlled development of ...,None,False
3,69411,2016,"Wrona, P, Rozanski, Z, Pach, G, Suponik, T, Po...",Closed coal mine shaft as a source of carbon d...,10.1007/s12665-016-5977-7,False
4,78334,2015,"Lyzwa, W., Wierzbowski, M., Olek, B., Stuchala...",The impact of energy policy 2030 on the edge c...,10.1109/EEM.2015.7216619,False


In [5]:
table = ws.get_all_values()
gdf = pd.DataFrame(table[1:], columns=table[0])
now = timezone.now().strftime("%Y-%m-%d %H:%M:%S")
for i, row in df.iterrows():
    if str(row['ID']) not in list(gdf['ID']):
        try:
            ws.append_row(list(row.values) + ["",now])
        except:
            print("Waiting...")
            time.sleep(100)
            ws.append_row(list(row.values) + ["",now])
    